# Model Inference

In [1]:
# imports
import torch
import matplotlib.pyplot as plt
import numpy as np
from source.net import *
from source.utils import *
from source.dataset import *
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [2]:
checkpoint = torch.load("/work/ai4bio2024/rxrx1/checkpoints/simclr_head/checkpoint4")
net_weights = checkpoint['model_state_dict']
device = "cuda:0"


In [3]:
backbone = load_net('simclr')
fc_head = load_net('fc_head', {'num_classes':1139})

net = CellClassifier(backbone, fc_head)
load_weights("/work/ai4bio2024/rxrx1/checkpoints/simclr_head/checkpoint4",net, 'cuda:0')
net = net.to(device)

In [4]:
split_sizes = [0.7, 0.15, 0.15]
dataset = Rxrx1('/work/ai4bio2024/rxrx1')
train_size = int(split_sizes[0] * len(dataset))
val_size = int(split_sizes[1] * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
test_dataloader = DataLoader(test_dataset, batch_size=128, pin_memory_device=device, pin_memory=True,
                             shuffle=True, num_workers=12, drop_last=True, prefetch_factor=2)

In [5]:
net.eval()
pred_label, true_label = [], []
for x_batch, _, y_batch in tqdm(test_dataloader):
        x_batch, y_batch = x_batch.to(torch.float).to(device), y_batch.to(device)
        y_pred = net(x_batch).argmax(dim=1)
        true_label.append(y_batch)
        pred_label.append(y_pred)


        

100%|██████████| 147/147 [00:15<00:00,  9.26it/s]


In [9]:
accuracy_score(torch.cat(true_label).cpu(), torch.cat(pred_label).cpu())

0.09183673469387756